In [ ]:
%load_ext autoreload

%autoreload 2

import zarr
from oxeo.water.models.pekel.pekel import PekelPredictor
from oxeo.core.models.tile import tile_from_id, TilePath
import matplotlib.pyplot as plt

import gcsfs
from oxeo.core.logging import logger
import zarr
fs = gcsfs.GCSFileSystem(project='oxeo-main')


In [ ]:
tile = tile_from_id("19_J_10000_73_697")
constellation = "landsat-8"
tile_path = TilePath(tile, constellation)

In [ ]:
predictor = PekelPredictor(fs, extra_not_used_arg="testing_extra_arg")

In [ ]:
import numpy as np
shape = zarr.open(fs.get_mapper(tile_path.data_path), "r").shape
print(shape[0])
masks = []
cnn_revisit_chunk_size = 10
for i in range(0, shape[0], cnn_revisit_chunk_size):
    logger.info(
        f"creating mask for {tile_path.path}, revisits {i} to {i + cnn_revisit_chunk_size}"
    )
    revisit_masks = predictor.predict(
        tile_path,
        revisit=slice(i, i + cnn_revisit_chunk_size),
    )
    masks.append(revisit_masks)
masks = np.vstack(masks)